## Strands Agents SDK

Introducing [Strands Agents](https://github.com/strands-agents/sdk-python), an Open Source AI Agents SDK! A simple-to-use, code-first framework that takes a model-driven approach to building and running AI agents in just a few lines of code, scaling from simple to complex use cases. The core of Strands is the simple agentic loop that connects the model and tools together, like two strands of DNA. It's already powering production AI agents in key AWS services like Amazon Q CLI, AWS Glue, and VPC Reachability Analyzer. It also includes 20+ built-in tools with support for thousands of Model Context Provider (MCP) servers. We're excited that Accenture, Anthropic, Meta, and others are joining us with support and contributions

## Use Case
We are going to setup a simple restaurant reservation agent that access to simple tools

![Agent Architecture](images/strands_agent.png)

## Notebook walk-through

In this notebook we will:
- Create agent using Strands Agents SDK
- Understand Strands session handlign
- Look at agent metrics
- Test the agent invocation

In [ ]:
%store -r

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import current_time
import uuid
import boto3
import json
import pandas as pd
from agent import create_dynamodb

In [ ]:
dynamodb = boto3.resource('dynamodb')
table_name = 'restaurant_bookings'
create_dynamodb(table_name)
table = dynamodb.Table(table_name)

In [ ]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [ ]:
@tool
def get_booking_details(booking_id: str) -> dict:
    """
    Retrieve the details of a specific restaurant booking using its unique identifier.

    This function queries the DynamoDB table to fetch the complete information
    associated with a given booking ID. It's useful for retrieving the full
    details of a reservation, including date, name, hour, and number of guests.

    Args:
        booking_id (str): The unique identifier of the booking to retrieve.
                          This should be a string, typically an 8-character UUID.

    Returns:
        dict: A dictionary containing the booking details if found. The structure includes:
              - booking_id (str): The unique identifier of the booking
              - date (str): The date of the booking in YYYY-MM-DD format
              - name (str): The name associated with the reservation
              - hour (str): The time of the booking in HH:MM format
              - num_guests (int): The number of guests for the booking
              
              If no booking is found, it returns a dictionary with a 'message' key
              indicating that no booking was found.
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in accessing the DynamoDB table or processing the request.
                   The error is caught and returned in the response dictionary.

    Example:
        >>> get_booking_details("12345678")
        {'booking_id': '12345678', 'date': '2023-05-15', 'name': 'John Doe', 'hour': '19:30', 'num_guests': 4}
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        print(e)
        return {'error': str(e)}

In [ ]:
@tool
def create_booking(date: str, name: str, hour: str, num_guests: int) -> dict:
    """
    Create a new restaurant booking and store it in the DynamoDB table.

    This function generates a unique booking ID and creates a new entry in the
    DynamoDB table with the provided booking details. It's used to make new
    reservations in the restaurant booking system.

    Args:
        date (str): The date of the booking in YYYY-MM-DD format.
        name (str): The name to identify the reservation. Typically the guest's name.
        hour (str): The time of the booking in HH:MM format.
        num_guests (int): The number of guests for the booking.

    Returns:
        dict: A dictionary containing the newly created booking ID if successful.
              The structure is:
              - booking_id (str): The unique identifier for the new booking (8-character UUID)
              
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in generating the UUID, accessing the DynamoDB table,
                   or processing the request. The error is caught and returned in the response dictionary.

    Example:
        >>> create_booking("2023-05-15", "John Doe", "19:30", 4)
        {'booking_id': 'a1b2c3d4'}
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        print(e)
        return {'error': str(e)}

In [ ]:
@tool
def delete_booking(booking_id: str) -> dict:
    """
    Delete an existing restaurant booking from the DynamoDB table.

    This function removes a booking entry from the database based on the provided
    booking ID. It's used to cancel reservations in the restaurant booking system.

    Args:
        booking_id (str): The unique identifier of the booking to delete.
                          This should be a string, typically an 8-character UUID.

    Returns:
        dict: A dictionary containing a message indicating the result of the operation.
              If successful, the structure is:
              - message (str): A success message with the deleted booking ID
              
              If the deletion fails (but doesn't raise an exception), it returns a
              dictionary with a message indicating the failure.
              
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in accessing the DynamoDB table or processing the request.
                   The error is caught and returned in the response dictionary.

    Example:
        >>> delete_booking("a1b2c3d4")
        {'message': 'Booking with ID a1b2c3d4 deleted successfully'}
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


In [ ]:
agent_instruction="""
##Role
You are a ABC Restaurant Booking agent. You are in charge of restaurant reservations.

##Instructions
Handle restaurant reservations inquiries and requests from users.

##Output Requirements
When responding to the end user, don't output your thinking steps.
Only give useful information to the end user.
"""


In [ ]:
model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    max_tokens=3000,
    temperature=1,
    top_p=1,
    additional_request_fields={
        "inferenceConfig": {
            "topK": 1,
        },
    }
)

agent = Agent(
    model=model,
    system_prompt=agent_instruction,
    tools=[current_time, get_booking_details, create_booking, delete_booking],
    callback_handler=None
)


In [ ]:
items = selectAllFromDynamodb()
items

### Test the agent

#### Create a booking

Let's start with creating a booking

In [ ]:
response = agent(
    """Hi, my name is Jane Doe.
    I want to book a table for 2 tommorow at 5pm.
    """
)
print(response)

Let's double check that the data was properly added to the dynamoDB table

In [ ]:
items = selectAllFromDynamodb()
items

### Sessions & State

Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Strands agents maintain state in several forms:

- Conversation History: The sequence of messages between user and agent
- Tool State: Information about tool executions and results
- Request State: Contextual information maintained within a single request

Now let's take a peak into the conversation history!

In [ ]:
print(json.dumps(agent.messages, indent=2))

### Metrics

Metrics are essential for understanding agent performance, optimizing behavior, and monitoring resource usage. The Strands Agents SDK provides comprehensive metrics tracking capabilities that give you visibility into how your agents operate.


In [ ]:
print(json.dumps(response.metrics.get_summary(), indent=2))

#### Check the booking details

Continue the conversation by asking about the booking details

In [ ]:
response = agent("Wait, can you get the details about the reservation I just made?")
print(response)

#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [ ]:
response = agent("Hmm, I changed my mind, can you cancel my reservation?")
print(response)

Check the dynamodb for verification

In [ ]:
items = selectAllFromDynamodb()
items